# Blob Classification with Tensorflow

In [ ]:
# This tutorial will give you a basic introduction for using machine learning for a classification problem
# Source code for this was in part from the following github repo:
# https://github.com/soerendip/Tensorflow-binary-classification/blob/master/Tensorflow-binary-classification-model.ipynb

## Import Libraries - Data Processing and Visualization

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Import Libraries - Vanilla Tensorflow

In [ ]:
# Using vanilla tensorflow will allow for a lower level creation of a model
# However, this means that the code will be more verbose

In [ ]:
# Can reduce load time by loading in specific modules
import tensorflow as tf


### Define constants for the script

In [ ]:
# Hyperparamters for training
LEARNING_RATE = 0.001
NUM_EPOCHS = 1000
BATCH_SIZE = 1
DISPLAY_STEP = 50


# Network Parameters; nodes per layer
n_input = 2
n_hidden_1 = 10 
n_hidden_2 = 10
n_hidden_3 = 10

# Output Num Classes
n_classes = 2 # Number of classes to predict

In [ ]:
# Get the data for the classification problem
# Use of a library method that allows us to get a random distribution of X group of points (blobs)

In [ ]:
data = make_blobs(n_samples=200,n_features=2,centers=2,random_state=101)

In [ ]:
# What the data consists of
data[0][:5] # X and Y coordinates

In [ ]:
data[1][:5] # Class label


In [ ]:
# Store the X and Y coordinates so they can be visually seen afterwards
features = data[0]

### Visualization of the problem

In [ ]:
plt.scatter(features[:,0],features[:,1])

In [ ]:
# get the class labels for the 2 groups of Blobs shown above
labels = data[1]
Y = np.array([labels, -(labels-1)]).T


In [ ]:
plt.scatter(features[:,0],features[:,1], c=labels,cmap='coolwarm')

### Setting up for Machine Learning

In [ ]:
# We begin with splitting the data into a train and test distribution, this is used to validate training progress
xtrain, xtest, ytrain, ytest = train_test_split(features, Y, test_size=0.2, random_state=101)
print(len(xtrain)) # how many training samples
print(len(xtest)) # how many validation samples

In [ ]:
xtrain.shape

In [ ]:
### Tensorflow Model for Training

In [ ]:
# Input for the tf Graph

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [ ]:
# Create the model

def feedforward_network(x, weights, biases):
    # Hidden Layer 1 - Relu Activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    # Hidden Layer 2 - Relu Activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # Hidden Layer 3 - Relu Activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    
    output_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return output_layer


In [ ]:
# Store Weights and bias terms for training
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [ ]:
# Construct model with function call
pred = feedforward_network(x, weights, biases)

In [ ]:
# Define the loss and optomizer for the graph
# reduce mean is to average the values
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph to begin training
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(NUM_EPOCHS):
        
        _, c = sess.run([optimizer, cost], feed_dict={x: xtrain,
                                                          y: ytrain})
        # Display logs per epoch step
        if epoch % DISPLAY_STEP == 0:
            print("Epoch:", '%04d' % (epoch), "cost=", "{:.9f}".format(c))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: xtest, y: ytest}))
    global result 
    result = tf.argmax(pred, 1).eval({x: xtest, y: ytest})

In [ ]:
result[:5]

In [ ]:
xtest[:5]

In [ ]:
ytest[:5]

## Validating the training

In [ ]:
# now we want to determine if the model is accurate by testing it out on some new data

In [ ]:
a,b,c = xtest[:,0], xtest[:,1], result

In [ ]:
assert (a.shape == b.shape == c.shape)
print("X, Y and Labels are all of same shape.")

In [ ]:
plt.scatter(a,b,s=50,c=c,cmap='Wistia')
plt.scatter(features[:,0],features[:,1], s=10,c=labels,cmap='coolwarm')
plt.show()

In [ ]:
# If you play around with the values for learning rate and num epochs, you can see the colors highlighted change
# color, this is due to the netowrk not being trained well so the predictions that it makes are not correct.
# In my version, the 2 groups with colors Orange and Yellow are in the 2 disctinct regions!!

## Congragulations on Completing your first ANN using Tensorflow!